# Aug 22 - 6S Sensitivity 

Scratch for testing sensitivity of 6S outputs to atmospheric parameters.

In [ ]:
import alive_progress
import matplotlib.pyplot as plt
import numpy as np
import rtm_wrapper.parameters as rtm_param
import rtm_wrapper.plot as rtm_plot
from rtm_wrapper.engines.sixs import PySixSEngine, pysixs_default_inputs
from rtm_wrapper.execution import ConcurrentExecutor
from rtm_wrapper.simulation import SweepSimulation

## Define input parameter ranges. 

In [ ]:
wavelengths = np.arange(0.4, 0.7, 0.0025)
# wavelengths = np.arange(0.58, 0.6, 0.0025)
# wavelengths = np.arange(0.62, 0.64, 0.0025)
# wavelengths = np.arange(0.65, 0.66, 0.0025)
ozone_range = (0.25, 0.45)  # cm-atm
water_range = (1, 4)  # g/cm^2
aot_range = (0.05, 0.5)

## Run simulations

In [ ]:
sweeps = {
    "ozone": SweepSimulation(
        {
            "wavelength.value": wavelengths,
            "atmosphere.ozone": np.arange(*ozone_range, 0.025),
        },
        base=pysixs_default_inputs().replace(
            atmosphere=rtm_param.AtmosphereWaterOzone(water=np.mean(water_range))
        ),
    ),
    "water": SweepSimulation(
        {
            "wavelength.value": wavelengths,
            "atmosphere.water": np.arange(*water_range, 0.25),
        },
        base=pysixs_default_inputs().replace(
            atmosphere=rtm_param.AtmosphereWaterOzone(ozone=np.mean(ozone_range))
        ),
    ),
    "aot": SweepSimulation(
        {
            "wavelength.value": wavelengths,
            "aerosol_profile.aot": np.arange(*aot_range, 0.05),
        },
        base=pysixs_default_inputs().replace(
            aerosol_profile=rtm_param.AerosolAOTSingleLayer(
                profile="Maritime",
                height=100,
            ),
        ),
    ),
    "altitude": SweepSimulation(
        {
            "wavelength.value": wavelengths,
            "altitude_target.value": [*np.logspace(-1, np.log10(81), 10)],
        },
        base=pysixs_default_inputs().replace(
            aerosol_profile=rtm_param.AerosolAOTSingleLayer(
                profile="Maritime",
                aot=0.3,
                height=100,
            ),
            altitude_target=rtm_param.AltitudeKilometers(),
        ),
    ),
}

engine = PySixSEngine()
runner = ConcurrentExecutor()

results = {}
for name, sweep in sweeps.items():
    with alive_progress.alive_bar(sweep.sweep_size, force_tty=True) as bar:
        runner.run(sweep, engine, step_callback=lambda _: bar())
    results[name] = runner.collect_results()

## Plot results

In [ ]:
fig, axs = plt.subplots(nrows=len(results), figsize=(8, 20))

for (name, output), ax in zip(results.items(), axs.flat):
    ax.set_prop_cycle(
        color=plt.cm.viridis(np.linspace(0, 1, list(output.sizes.values())[-1]))
    )

    rtm_plot.plot_sweep_legend(
        output.total_transmission,
        ax=ax,
        xaxis_dim="wavelength.value",
        legend_kwargs={"loc": "lower left", "ncol": 2},
    )
    ax.set_title(f"{name.capitalize()} Sensitivity")
    ax.grid(True)